In [1]:
#Importing libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [2]:
#loading the train data and test data
train=pd.read_csv("Dataset/20percent_labelled.csv")
test=pd.read_csv("Dataset/Test.csv")

# Data Preprocessing

In [10]:
train.replace([np.inf, -np.inf], np.nan, inplace=True)

In [12]:
train=train.dropna(subset=['Service'])

In [14]:
#finding categoritcal features
numeric_columns=train._get_numeric_data().columns
categorical_columns=list(set(train.columns)-set(numeric_columns))
categorical_columns.remove('Result')
categorical_columns

['Protocol_type', 'Service', 'Flag']

In [25]:
#analyzing the correlation,since Num_root and Num_compromised are highly correlated, so Num_root should be ignored during analysis
train.drop('Num_root',axis=1,inplace=True)

In [27]:
#highly correlated with serror_rate, so should be ignored(0.9983615072725952)

train.drop('Srv_serror_rate',axis=1,inplace=True)

In [29]:
#rerror_rate highly correlated with srv_error_rate,so srv_error_rate needs to be ignored(0.9890076529820789)
train.drop('Srv_error_rate',axis=1,inplace=True)

In [30]:
#Dst_host_srv_serror_rate is high correlated with srv_serror_rate, so needs to be ignored.(0.9993041091850098)
train.drop('Dst_host_srv_s error_rate',axis=1,inplace=True)

In [31]:
#Dst_host_serror_rate is high correlated with rerror_rate, so needs to be ignored.
train.drop('Dst_host_serro r_rate',axis=1,inplace=True)

In [33]:
#Dst_host_srv_rerror_rate is high correlated with Rerror_rate, so needs to be ignored.(0.964449764823)
train.drop('Dst_host_srv_r error_rate',axis=1,inplace=True)

In [34]:
#Dst_host_rerror_rate is high correlated with srv_error_rate, so needs to be ignored.(0.917822)
train.drop('Dst_host_rerro r_rate',axis=1,inplace=True)

In [35]:
#Dst_host_same\n_srv_rate is highly correlated with Dst_host_srv_count and should be ignored.(0.896663)
train.drop('Dst_host_same\n_srv_rate',axis=1,inplace=True)

In [36]:
train.drop('Num_outbound_cmds',axis=1,inplace=True)

In [42]:
#converting categorical values into numeric 
#can be done in this way as well
#from sklearn import preprocessing
#label_encoder=preprocessing.LabelEncoder()
#train['Flag']=label_encoder.fit_transform(train['Flag'])
train.Protocol_type=train.Protocol_type.map({"tcp":0,"udp":1,"icmp":2})
train.Service=train.Service.map({"ftp_data":0,"other":1,"private":2,"http":3,"remote_job":4,"name":5,"netbios_dgm":6,"eco_i":7,"mtp":8,"telnet":9,"finger":10,"domain_u":11,"supdup":12,"uucp_path":13,"Z39_50":14,"smtp":15,"csnet_ns":16,"uucp":17,"netbios_dgm":18,"urp_i":19,"auth":20,"domain":21,"ftp":22,"bgp":23,"ldap":24,"ecr_i":25,"gopher":26,"vmnet":27,"systat":28,"http_433":29,"efs":30,"whois":31,"imap4":32,"iso_tsap":33,"echo":34,"klogin":35,"link":36,"sunrpc":37,"login":38,"kshell":39,"sql_net":40,"time":41,"hostnames":42,"exec":43,"ntp_u":44,"discard":45,"nntp":46,"courier":47,"ctf":48,"ssh":49,"daytime":50,"shell":51,"netstat":52,"pop3":53,"nnsp":54,"IRC":55,"pop_2":56,"printer":57,"tim_i":58,"pm_dump":59,"red_i":60,"netbios_ssn":61,"rje":62,"X11":63,"urh_i":64,"http_8001":65,"aol":66,"http_2784":67,"tftp_u":68,"harvest":69})
train.Flag=train.Flag.map({"SF":0,"S0":1,"REJ":2,"RSTR":3,"SH":4,"RSTO":5,"S1":6,"RSTOS0":7,"S3":8,"S2":9,"OTH":10})

# Model Building

In [44]:
predictors = train[train.columns.difference(['Result'])]

In [45]:
#allocating the targeted columns
X=predictors.values
#assigning target variable
Y=train.Result

In [46]:
#using train_test_split function from sklearn to divide the train dataset
#importing library
from sklearn.model_selection import train_test_split

In [47]:
#split the data into train and cross validation 
#same train and test sets across different executions hence random_state=0.
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=0)

In [48]:
#importing decision tree and accuracy_score from sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn import metrics

In [49]:
X_train = np.nan_to_num(X_train.astype(np.float32))
X_test = np.nan_to_num(X_test.astype(np.float32))

In [50]:

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [51]:
#Stacked Model
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [52]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score





In [56]:
from sklearn.ensemble import BaggingClassifier
bag_model=BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True
)
bag_model.fit(X_train,y_train)
bag_model.oob_score_

0.9954633095157083

In [57]:
bag_model.score(X_test,y_test)

0.9964276263561789

In [58]:
scores=cross_val_score(bag_model,X_train,y_train)
scores.mean()


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0.9943290724933546